## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

## Prepare data for model training

In [2]:
with open("../input/tps-may-data-preprocess-v5/TPS_May_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

del processed_data
gc.collect()

0

In [3]:
cat_cols = ['feature_0','feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7',
            'feature_8','feature_9','feature_10','feature_11','feature_12','feature_13','feature_14','feature_15',
            'feature_16','feature_17','feature_18','feature_19','feature_20','feature_21','feature_22','feature_23',
            'feature_24','feature_25','feature_26','feature_27','feature_28','feature_29','feature_30','feature_31',
            'feature_32','feature_33','feature_34','feature_35','feature_36','feature_37','feature_38','feature_39',
            'feature_40','feature_41','feature_42','feature_43','feature_44','feature_45','feature_46','feature_47',
            'feature_48','feature_49','clusters__0','clusters__1','clusters__2','clusters__3','clusters__4',
            'clusters__5','clusters__6','clusters__7','clusters__8','clusters__9','clusters__10','clusters__11',
            'clusters__12','clusters__13','clusters__14']

train_df[cat_cols] = train_df[cat_cols].astype(int)
test_df[cat_cols] = test_df[cat_cols].astype(int)
cat_cols_indices = [train_df.columns.get_loc(col) for col in cat_cols]
print(cat_cols_indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [4]:
Xtrain = train_df.loc[:, train_df.columns != 'target'].values
Ytrain = train_df['target'].values
Ytrain_oh = pd.get_dummies(train_df['target']).values
Xtest = test_df.values

print("Xtrain: {} \nYtrain: {} \nYtrain_oh: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, 
                                                                   Ytrain_oh.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (99918, 100) 
Ytrain: (99918,) 
Ytrain_oh: (99918, 4) 
Xtest: (50000, 100)


20

## Build and validate the model

In [5]:
params = {}
params["objective"] = 'multiclass'
params['metric'] = 'multi_logloss'
params['boosting'] = 'gbdt'
#params["device_type"] = 'gpu'
params['num_class'] = 4
params['is_unbalance'] = True
params["learning_rate"] = 0.012
params["lambda_l2"] = 0.0256
params["num_leaves"] = 52
#params["max_depth"] = 10
params["feature_fraction"] = 0.503
params["bagging_fraction"] = 0.6741
params["bagging_freq"] = 8
params["bagging_seed"] = 10
params["min_data_in_leaf"] = 10
params["verbosity"] = -1
num_rounds = 5000

In [6]:
FOLD = 10
NUM_SEED = 3

# Prediction Clipping Thresholds
p_min = 0.025
p_max = 1 - p_min

np.random.seed(2021)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_lgb = np.zeros((Ytrain.shape[0], 4))
y_pred_final_lgb = np.zeros((Xtest.shape[0], 4))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain, Ytrain)):
        counter += 1

        train_x, train_y, train_y_oh = Xtrain[train], Ytrain[train], Ytrain_oh[train]
        val_x, val_y, val_y_oh = Xtrain[val], Ytrain[val], Ytrain_oh[val]

        lgtrain = lgb.Dataset(train_x, label=train_y.ravel())
        lgvalidation = lgb.Dataset(val_x, label=val_y.ravel())

        model = lgb.train(params, lgtrain, num_rounds, 
                          valid_sets=[lgtrain, lgvalidation], 
                          categorical_feature=cat_cols_indices,
                          early_stopping_rounds=200, verbose_eval=100)

        y_pred = model.predict(val_x, num_iteration=model.best_iteration)
        y_pred = np.clip(y_pred, p_min, p_max)
        y_pred_meta_lgb[val] += y_pred
        y_pred_final_lgb += model.predict(Xtest, num_iteration=model.best_iteration)
        
        score = log_loss(val_y_oh, y_pred)
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_lgb = y_pred_meta_lgb / float(NUM_SEED)
y_pred_final_lgb = y_pred_final_lgb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.07624	valid_1's multi_logloss: 1.10298
[200]	training's multi_logloss: 1.04579	valid_1's multi_logloss: 1.09702
[300]	training's multi_logloss: 1.02031	valid_1's multi_logloss: 1.09448
[400]	training's multi_logloss: 0.997913	valid_1's multi_logloss: 1.09379
[500]	training's multi_logloss: 0.977758	valid_1's multi_logloss: 1.0936
[600]	training's multi_logloss: 0.958772	valid_1's multi_logloss: 1.09398
[700]	training's multi_logloss: 0.940914	valid_1's multi_logloss: 1.09466
Early stopping, best iteration is:
[508]	training's multi_logloss: 0.976205	valid_1's multi_logloss: 1.09351
Seed-85 | Fold-0 | OOF Score: 1.0935144558843868


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.07661	valid_1's multi_logloss: 1.10196
[200]	training's multi_logloss: 1.04632	valid_1's multi_logloss: 1.09517
[300]	training's multi_logloss: 1.02105	valid_1's multi_logloss: 1.0921
[400]	training's multi_logloss: 0.998635	valid_1's multi_logloss: 1.09087
[500]	training's multi_logloss: 0.978247	valid_1's multi_logloss: 1.09057
[600]	training's multi_logloss: 0.959476	valid_1's multi_logloss: 1.0907
[700]	training's multi_logloss: 0.941556	valid_1's multi_logloss: 1.09083
Early stopping, best iteration is:
[515]	training's multi_logloss: 0.975413	valid_1's multi_logloss: 1.09044
Seed-85 | Fold-1 | OOF Score: 1.0904386618211377


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


...

Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.07633	valid_1's multi_logloss: 1.10307
[200]	training's multi_logloss: 1.04602	valid_1's multi_logloss: 1.09715
[300]	training's multi_logloss: 1.02064	valid_1's multi_logloss: 1.09494
[400]	training's multi_logloss: 0.998154	valid_1's multi_logloss: 1.09445
[500]	training's multi_logloss: 0.977703	valid_1's multi_logloss: 1.09422
[600]	training's multi_logloss: 0.958745	valid_1's multi_logloss: 1.09487
Early stopping, best iteration is:
[449]	training's multi_logloss: 0.987878	valid_1's multi_logloss: 1.09412
Seed-0 | Fold-7 | OOF Score: 1.0941202431824437


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.07644	valid_1's multi_logloss: 1.10257
[200]	training's multi_logloss: 1.04585	valid_1's multi_logloss: 1.09675
[300]	training's multi_logloss: 1.02053	valid_1's multi_logloss: 1.09448
[400]	training's multi_logloss: 0.998134	valid_1's multi_logloss: 1.0937
[500]	training's multi_logloss: 0.977789	valid_1's multi_logloss: 1.09365
[600]	training's multi_logloss: 0.958796	valid_1's multi_logloss: 1.09367
Early stopping, best iteration is:
[457]	training's multi_logloss: 0.986335	valid_1's multi_logloss: 1.0934
Seed-0 | Fold-8 | OOF Score: 1.0933972251592365


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.07627	valid_1's multi_logloss: 1.10441
[200]	training's multi_logloss: 1.04586	valid_1's multi_logloss: 1.0988
[300]	training's multi_logloss: 1.02055	valid_1's multi_logloss: 1.09656
[400]	training's multi_logloss: 0.998222	valid_1's multi_logloss: 1.0957
[500]	training's multi_logloss: 0.977735	valid_1's multi_logloss: 1.09564
[600]	training's multi_logloss: 0.959009	valid_1's multi_logloss: 1.09598
Early stopping, best iteration is:
[488]	training's multi_logloss: 0.980111	valid_1's multi_logloss: 1.09557
Seed-0 | Fold-9 | OOF Score: 1.0955747155449183

Seed: 0 | Aggregate OOF Score: 1.0928278674292717


Aggregate OOF Score: 1.092921063108014


In [7]:
np.savez_compressed('./LGB_Meta_Features.npz',
                    y_pred_meta_lgb=y_pred_meta_lgb, 
                    oof_score=oof_score,
                    y_pred_final_lgb=y_pred_final_lgb)

## Create submission file

In [8]:
y_pred_final_lgb = np.clip(y_pred_final_lgb, p_min, p_max)

test_df = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_lgb[:,0]
submit_df['Class_2'] = y_pred_final_lgb[:,1]
submit_df['Class_3'] = y_pred_final_lgb[:,2]
submit_df['Class_4'] = y_pred_final_lgb[:,3]
submit_df.head()

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.094618,0.582060,0.189509,0.133813
1,100001,0.067342,0.648476,0.178464,0.105718
2,100002,0.062800,0.682327,0.169452,0.085421
3,100003,0.081586,0.518184,0.311256,0.088974
4,100004,0.068587,0.674762,0.165719,0.090931


In [9]:
submit_df.to_csv("./LGB_submission.csv", index=False)